# Sample Summary Tables

In [12]:
library(dplyr)
library(tidyverse)
library(ggplot2)
library(tidyr)
library(purrr)
library(stringr)

## SCTLD
- bks sample set (see CBC_Metagenomics repo for more)

In [13]:
getwd()

[1] "/Users/brookesienkiewicz/Documents/Code_notebook/Data_Summaries"

In [14]:
# upload sequence list and rename col 
samplelist<-read.table('../inputs/genohublist_sctld2024.txt', header = FALSE)

In [15]:
nrow(samplelist)

[1] 222

In [16]:
# upload colony and sample data
colony <- read_csv("/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_ColonyData.csv", show_col_types = FALSE)
sctld_samples <- read.csv("/Users/brookesienkiewicz/Documents/sctld/SCTLD_samples/Sample_Data/CBC_samples.csv")

New names:
• `` -> `...1`


In [17]:
nrow(sctld_samples)

[1] 2529

In [18]:
# add colony ids to all dfs
add_colonyid <- function(df) {
    # get transect # 
    df$transect_id <- paste0('T',df$TransectNum)
    # make colony id 
    df$colony_id <- paste(df$transect_id, df$NewTagNum, df$Species,
                                 sep = "_")
    return(df)
}

# apply 
sctld_samples<-add_colonyid(sctld_samples)
colony <- add_colonyid(colony)

In [19]:
head(sctld_samples)

,Month_year,Country,Location,CollectionDate,Transect,TransectNum,OldTagNum,NewTagNum,Species,Time_sampled,⋯,SampleNum,Health_status,Sampling_notes,Tubelabel_species,Sample_physical_location,Extraction_physical_location,Date_sequenced,Notes,transect_id,colony_id
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,122022,BEL,CBC,12/5/22,CURLEW,4,,77,OFAV,,⋯,18,Healthy,,122022_BEL_CBC_T4_18_OFAV,,,,,T4,T4_77_OFAV
2,92023,BEL,CBC,9/27/23,CBC30N,1,,1,SSID,,⋯,185,Diseased_Margin,only margin sample available,092023_BEL_CBC_T1_185_SSID,UML_NARWHAL_R1_B10,,,,T1,T1_1_SSID
3,92023,BEL,CBC,9/25/23,CBC30N,1,,2,PAST,,⋯,171,Healthy,CLP 90%,092023_BEL_CBC_T1_171_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,T1,T1_2_PAST
4,92023,BEL,CBC,9/25/23,CBC30N,1,,3,SSID,,⋯,173,Healthy,CLP 80%; DC 20%,092023_BEL_CBC_T1_173_SSID,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,T1,T1_3_SSID
5,92023,BEL,CBC,9/25/23,CBC30N,1,,12,PSTR,,⋯,177,Healthy,No CL,092023_BEL_CBC_T1_177_PSTR,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,,T1,T1_12_PSTR
6,92023,BEL,CBC,9/25/23,CBC30N,1,,13,PAST,,⋯,175,Healthy,No CL,092023_BEL_CBC_T1_175_PAST,UML_NARWHAL_R1_B10,UML_NARWHAL_R2_B12,,R2_B15 EXTRACTED TWICE,T1,T1_13_PAST


In [20]:
#filter sample data to match sequence list 
sctld_samples<-sctld_samples %>%
    filter(Sample_type == 'Core_EtOH') %>%
    filter(Tubelabel_species %in% samplelist$V1)
nrow(sctld_samples)
# filter colony data to match sequence list 
sctld_colony <- colony %>%
    filter(colony_id %in% sctld_samples$colony_id)
length(unique(sctld_colony$colony_id))
# 113 unique colonies - 220 total samples 

[1] 220

[1] 113

In [21]:
# colony summary 
summary_sctld <- sctld_colony %>%
  group_by(Species) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)
summary_sctld

MCAV,MMEA,OANN,OFAV,PAST,PSTR
<int>,<int>,<int>,<int>,<int>,<int>
31,13,7,11,20,31


In [22]:
# sample summary 
total_samples <- sctld_samples %>%
  group_by(Species) %>%
  summarise(n = n(), .groups = "drop") %>%
  pivot_wider(names_from = Species, values_from = n, values_fill = 0)
total_samples

MCAV,MMEA,OANN,OFAV,PAST,PSTR
<int>,<int>,<int>,<int>,<int>,<int>
68,13,14,18,53,54


In [34]:
# convert collection date to date format 

sctld_samples$dates <- as.Date(sctld_samples$CollectionDate, format = "%m/%d/%y")

# convert to mmyyyy and make separate column for written month year 
sctld_samples$Month_year <- format(sctld_samples$dates, "%b %Y")

# make all 2019 same month for simplicity 
sctld_samples[sctld_samples$Month_year=='Oct 2019',"Month_year"]='Jun 2019'
unique(sctld_samples$Month_year)

# arrange in chronological order 
sctld_samples$Month_year<-factor(sctld_samples$Month_year, levels = c("Jun 2019","May 2022","Dec 2022")) 

[1] "Dec 2022" "May 2022" "Jun 2019"

In [ ]:
# make sample table - test one species first 
# for mcav: 
    # how many unique colonies were sampled each year 
## can use sample data - unique colony id per sample date?
    # how many samples were taken each year 

In [61]:
# filter for mcav - to test a subset first 
test_mcav <- sctld_samples %>%
    filter(Species == 'MCAV') %>%
    select(c('Month_year','Health_status','Tubelabel_species','colony_id'))
# arrange in chronological order 
test_mcav$Month_year<-factor(test_mcav$Month_year, levels = c("Jun 2019","May 2022","Dec 2022")) 

In [79]:
head(test_mcav)

,Month_year,Health_status,Tubelabel_species,colony_id
,<fct>,<chr>,<chr>,<chr>
1,Dec 2022,Diseased_Tissue,122022_BEL_CBC_T1_144_MCAV,T1_8_MCAV
2,Dec 2022,Healthy,122022_BEL_CBC_T1_151_MCAV,T1_24_MCAV
3,Dec 2022,Healthy,122022_BEL_CBC_T1_153_MCAV,T1_7_MCAV
4,Dec 2022,Healthy,122022_BEL_CBC_T2_85_MCAV,T2_59_MCAV
5,Dec 2022,Healthy,122022_BEL_CBC_T2_86_MCAV,T2_60_MCAV
6,Dec 2022,Healthy,122022_BEL_CBC_T2_88_MCAV,T2_55_MCAV


In [70]:
# each row is a unique sample - 68 samples total 
# unique num of colonies at each sample date... pivot first? 
sorted_mcav<-test_mcav %>%
    pivot_wider(names_from = Month_year,
               values_from = Health_status,
               names_sort = TRUE) %>%
    arrange(colony_id)
head(sorted_mcav)

Tubelabel_species,colony_id,Jun 2019,May 2022,Dec 2022
<chr>,<chr>,<chr>,<chr>,<chr>
052022_BEL_CBC_T1_39_MCAV,T1_14_MCAV,NA,Diseased_Margin,NA
052022_BEL_CBC_T1_60_MCAV,T1_14_MCAV,NA,Diseased_Tissue,NA
062019_BEL_CBC_T1_17_MCAV,T1_14_MCAV,Healthy,NA,NA
052022_BEL_CBC_T1_40_MCAV,T1_15_MCAV,NA,Diseased_Margin,NA
052022_BEL_CBC_T1_70_MCAV,T1_15_MCAV,NA,Diseased_Tissue,NA
062019_BEL_CBC_T1_24_MCAV,T1_15_MCAV,Healthy,NA,NA


In [78]:
sum(!is.na(sorted_mcav$`Jun 2019`))
sum(!is.na(sorted_mcav$`May 2022`))
sum(!is.na(sorted_mcav$`Dec 2022`))

[1] 24

[1] 22

[1] 22

In [88]:
sorted_mcav[sorted_mcav$colony_id=='T1_14_MCAV',]

Tubelabel_species,colony_id,Jun 2019,May 2022,Dec 2022
<chr>,<chr>,<chr>,<chr>,<chr>
052022_BEL_CBC_T1_39_MCAV,T1_14_MCAV,NA,Diseased_Margin,NA
052022_BEL_CBC_T1_60_MCAV,T1_14_MCAV,NA,Diseased_Tissue,NA
062019_BEL_CBC_T1_17_MCAV,T1_14_MCAV,Healthy,NA,NA


In [84]:
# how many colonies do we have pre and post samples for? 
for (colony in list(unique(sorted_mcav$colony_id))){
    print(colony)
    df<-sorted_mcav[sorted_mcav$colony_id==colony,]
    mutate(Pre = case_when(
        !is.na(`Jun 2019`) ~ Healthy 
    )
    }

 [1] "T1_14_MCAV"  "T1_15_MCAV"  "T1_24_MCAV"  "T1_329_MCAV" "T1_333_MCAV"
 [6] "T1_342_MCAV" "T1_355_MCAV" "T1_7_MCAV"   "T1_8_MCAV"   "T2_53_MCAV" 
[11] "T2_55_MCAV"  "T2_56_MCAV"  "T2_59_MCAV"  "T2_60_MCAV"  "T2_61_MCAV" 
[16] "T2_69_MCAV"  "T3_12_MCAV"  "T3_14_MCAV"  "T3_15_MCAV"  "T3_17_MCAV" 
[21] "T3_21_MCAV"  "T3_22_MCAV"  "T3_2_MCAV"   "T3_67_MCAV"  "T3_71_MCAV" 
[26] "T3_9_MCAV"   "T4_28_MCAV"  "T4_30_MCAV"  "T4_76_MCAV"  "T4_94_MCAV" 
[31] "T4_95_MCAV" 


Warning message in sorted_mcav$colony_id == colony:
“longer object length is not a multiple of shorter object length”


In [ ]:
# sankay plot for which colonies we have samples for??? 

In [ ]:
# repeat for colony data 


In [79]:
colnames(sctld_colony)
# pivot 

[1] "...1"                     "Date_InitialTag"         
 [3] "Transect"                 "TransectNum"             
 [5] "OldTagNum"                "NewTagNum"               
 [7] "Species"                  "Meter"                   
 [9] "Meters_90"                "Direction"               
[11] "Size_Class"               "MaxDiameter"             
[13] "Height"                   "Date_DocumentedDisease"  
[15] "Date_DocumentedMortality" "Notes_062019"            
[17] "062019_Condition"         "062019_Percentage"       
[19] "Notes_052022"             "052022_Condition"        
[21] "052022_Percentage"        "Notes_122022"            
[23] "122022_Condition"         "122022_Percentage"       
[25] "Notes_092023"             "092023_Condition"        
[27] "092023_Percentage"        "Notes_112023"            
[29] "112023_Condition"         "112023_Percentage"       
[31] "Notes_122023"             "122023_Condition"        
[33] "122023_Percentage"        "Notes_012024"            
[35] "012024_Condition"         "012024_Percentage"       
[37] "Notes_022024"             "022024_Condition"        
[39] "022024_Percentage"        "Notes_042024"            
[41] "042024_Condition"         "042024_Percentage"       
[43] "062024_Condition"         "062024_Percentage"       
[45] "Notes_062024"             "082024_Condition"        
[47] "082024_Percentage"        "Notes_082024"            
[49] "122024_Condition"         "122024_Percentage"       
[51] "Notes_122024"             "062025_Condition"        
[53] "062025_Percentage"        "Notes_062025"            
[55] "immune_y/n"               "checked_colonies"        
[57] "transect_id"              "colony_id"